# Benchmarking Custom Circuits in Tour-de-Gross

This notebook gives a convenient workflow benchmarking custom circuits in PBC form. This relies on the `bicycle_compiler` and `bicycle_numerics` crates.

Before running this notebook, make sure the rust code is compiled by running `cargo build --release` in the root directory.


In [1]:
import asyncio, os, json
import matplotlib.pyplot as plt
from typing import Literal

async def run_command(*cmdlist:list[str], input_data=None):
    """Run the exectuable once for one set of input parameters. Supports streaming data from stdin."""

    process = await asyncio.create_subprocess_exec(
        *[str(x) for x in cmdlist],
        stdin=asyncio.subprocess.PIPE if input_data is not None else None,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    stdout, stderr = await process.communicate(
            input=input_data.encode("utf-8") if input_data is not None else None
        )
    if stderr:
        print(f"[stderr]\n{stderr.decode()}")
    return stdout.decode()

def pretty_print_compiled_circuit(string, max_lines=None):
    """Prints a circuit as compiled by bicycle_compiler. Truncates to the first few lines if desired."""

    lines = string.replace("],[","]\n[").split("\n")
    if max_lines is not None:
        lines = lines[:max_lines]
    for line in lines:
        print(line)

## Generate measurement tables

Measurement tables encode optimized circuits for how to address particular Pauli matrices within a code block, see section 3.2 of [Tour de Gross (2506.03094)](https://arxiv.org/abs/2506.03094) and the [Compiler Crate Readme](../crates/bicycle_compiler/README.md).

The gross and two-gross codes have different LPUs and have a different set of Pauli matrices that are easy to synthesize directly. Hence a code must be specified.

In [2]:
code = "gross" # or "two-gross"

In [3]:
measurement_table_path = f"table_{code}.dat"

if not os.path.exists(measurement_table_path):
    print("Generating ", measurement_table_path)
    await run_command("../target/release/bicycle_compiler",
                    f"{code}", "generate",
                    measurement_table_path)
else:
    print(measurement_table_path, "already exists.")

table_gross.dat already exists.


## Compile Custom Circuits

Given a circuit in PBC form, with each rotation or measurement, in JSON list format, `bicycle_compiler` outputs a compiled circuit using the Bicycle ISA instruction set.

See the [Compiler Crate Readme](../crates/bicycle_compiler/README.md) for more information.

In [4]:
async def compile_pbc_circuit(circuit:str, code_:Literal["gross", "two-gross"]):
    # Takes a circuit in PBC form as a json list representation
    # Outputs a Bicycle ISA circuit as a json list

    return await run_command("../target/release/bicycle_compiler",
                                code_,
                                "--measurement-table", f"table_{code_}.dat",
                             input_data=circuit)

In [5]:
test_circuit = """{"Rotation":{"basis":["X","X","I","I","I","I","I","I","I","I","I","Y"],"angle":"0.125"}}
{"Rotation":{"basis":["Z","Z","I","I","I","I","I","I","I","I","I","I"],"angle":"0.5"}}
{"Rotation":{"basis":["X","X","I","I","I","I","I","I","I","I","I","I"],"angle":"-0.125"}}
{"Measurement":{"basis":["Z","X","I","I","I","I","I","I","I","I","I","I"],"flip_result":true}}
{"Measurement":{"basis":["X","I","I","I","I","Z","I","I","I","I","I","I"],"flip_result":false}}
"""

pretty_print_compiled_circuit(await compile_pbc_circuit(test_circuit, code), max_lines=20)

[[[0,{"Measure":{"p1":"X","p7":"I"}}]]
[[0,{"Automorphism":{"x":5,"y":1}}]]
[[0,{"Measure":{"p1":"Y","p7":"I"}}]]
[[0,{"Automorphism":{"x":1,"y":5}}]]
[[0,{"Measure":{"p1":"Y","p7":"I"}}]]
[[1,{"Measure":{"p1":"X","p7":"I"}}]]
[[1,{"Automorphism":{"x":5,"y":1}}]]
[[1,{"Measure":{"p1":"Z","p7":"I"}}]]
[[1,{"Automorphism":{"x":1,"y":5}}]]
[[1,{"Measure":{"p1":"Y","p7":"I"}}]]
[[1,{"Measure":{"p1":"X","p7":"I"}}]]
[[1,{"Automorphism":{"x":5,"y":3}}]]
[[1,{"Measure":{"p1":"Z","p7":"I"}}]]
[[1,{"Automorphism":{"x":1,"y":3}}]]
[[1,{"Measure":{"p1":"Y","p7":"I"}}]]
[[0,{"Automorphism":{"x":5,"y":1}}]]
[[0,{"Measure":{"p1":"Z","p7":"I"}}]]
[[0,{"Automorphism":{"x":1,"y":5}}]]
[[1,{"Automorphism":{"x":0,"y":5}}]]
[[1,{"Measure":{"p1":"Y","p7":"I"}}]]


In [6]:
def transverse_field_ising_model(nqubits:int, ntrotter:int, theta_ZZ:float, theta_X:float):
    gates = []

    for _ in range(ntrotter):
        # exp(i theta_ZZ Z_i Z_{i+1}) for all adjacent pairs of qubits
        for i in range(nqubits-1):
            pauli = ["Z" if j in [i,i+1] else "I" for j in range(nqubits)]
            gates.append({"Rotation":{"basis":pauli, "angle":str(theta_ZZ)}})

        # exp(i theta_X X_i) for all qubits
        for i in range(nqubits):
            pauli = ["X" if j == i else "I" for j in range(nqubits)]
            gates.append({"Rotation":{"basis":pauli, "angle":str(theta_X)}})

    return "\n".join(map(json.dumps,gates)).replace(" ","")

In [7]:
pretty_print_compiled_circuit(
    await compile_pbc_circuit(transverse_field_ising_model(20, 5, 0.1, 0.1), code),
    max_lines=20
)

[[[0,{"Measure":{"p1":"X","p7":"I"}}]]
[[0,{"Automorphism":{"x":4,"y":5}}]]
[[0,{"Measure":{"p1":"Z","p7":"I"}}]]
[[0,{"Automorphism":{"x":2,"y":1}}]]
[[0,{"Measure":{"p1":"Y","p7":"I"}}]]
[[0,{"Measure":{"p1":"X","p7":"I"}}]]
[[0,{"Automorphism":{"x":2,"y":1}}]]
[[0,{"Measure":{"p1":"Y","p7":"I"}}]]
[[0,{"Automorphism":{"x":4,"y":5}}]]
[[0,{"Measure":{"p1":"Z","p7":"I"}}]]
[[0,{"Measure":{"p1":"Y","p7":"I"}}]]
[[1,{"Measure":{"p1":"Y","p7":"I"}}]]
[[0,{"Automorphism":{"x":2,"y":1}}]]
[[0,{"Measure":{"p1":"X","p7":"I"}}]]
[[0,{"Automorphism":{"x":4,"y":5}}]]
[[0,{"JointMeasure":{"p1":"Z","p7":"I"}}]
[1,{"JointMeasure":{"p1":"Z","p7":"I"}}]]
[[1,{"TGate":{"basis":"Z","primed":false,"adjoint":true}}]]
[[1,{"TGate":{"basis":"X","primed":false,"adjoint":true}}]]
[[1,{"TGate":{"basis":"Z","primed":false,"adjoint":true}}]]


### Compile Qiskit circuits

_Note: This part requires Qiskit v2.3 or later to be installed._

Qiskit can compile circuits into PBC format, which we can then parse and compile to Gross code ISA using `bicycle_compiler`. The `scripts/qiskit_parser.py` module provides a parser to iterate over Qiskit circuits and yield the JSON list format required by the `bicycle_compiler`. See also `scripts/qiskit_demo.sh` for an example running a larger circuit.

In [8]:
import qiskit

if qiskit.__version__ < "2.3.0":
    print(f"This cell requires Qiskit v2.3 or later to be installed, current version {qiskit.__version__}")
else:
    import json
    from qiskit import QuantumCircuit, transpile
    from qiskit.circuit.library import PauliEvolutionGate
    from qiskit.transpiler.passes import LitinskiTransformation
    from qiskit.quantum_info import SparseObservable

    # use the parser script from the local scripts folder 
    import sys
    sys.path.insert(0, "../scripts")
    from qiskit_parser import iter_qiskit_pbc_circuit

    def build_evolution_circuit(num_qubits: int, reps: int) -> QuantumCircuit:
        """Build a circuit to compile to Gross code ISA."""
        obs = SparseObservable.from_sparse_list(
            [
                (inter, [i, i + 1], -1)
                for inter in ("XX", "YY", "ZZ")
                for i in range(num_qubits - 1)
            ]
            + [("Z", [i], 0.5) for i in range(num_qubits)],
            num_qubits=num_qubits,
        )
        evo = PauliEvolutionGate(obs, time=1 / reps)
    
        circuit = QuantumCircuit(num_qubits, num_qubits)
        for _ in range(reps):
            circuit.append(evo, circuit.qubits)
    
        for i, _ in enumerate(circuit.qubits):
            circuit.measure(i, i)
    
        return circuit

    # build the circuit
    circuit = build_evolution_circuit(10, 10)

    # transpile it to PBC, by first going to Clifford+RZ, then
    # applying the Litinski transformation
    tqc = transpile(circuit, basis_gates=["h", "sx", "sxdg", "rz", "cx"])
    pbc = LitinskiTransformation(fix_clifford=False)(tqc)
    pbc_iter = iter_qiskit_pbc_circuit(pbc)
    
    # for large scale circuits, iterate by piping the 
    # output directly into the compiler
    circuit_str = "\n".join(map(json.dumps, pbc_iter))
    pretty_print_compiled_circuit(
        await compile_pbc_circuit(circuit_str, code),
        max_lines=20
    )

[[[0,{"Measure":{"p1":"X","p7":"I"}}]]
[[0,{"Automorphism":{"x":5,"y":1}}]]
[[0,{"Measure":{"p1":"Y","p7":"I"}}]]
[[0,{"Automorphism":{"x":1,"y":5}}]]
[[0,{"Measure":{"p1":"Y","p7":"I"}}]]
[[0,{"Automorphism":{"x":5,"y":1}}]]
[[0,{"Measure":{"p1":"Z","p7":"I"}}]]
[[0,{"Automorphism":{"x":1,"y":5}}]]
[[0,{"TGate":{"basis":"Z","primed":false,"adjoint":false}}]]
[[0,{"TGate":{"basis":"X","primed":false,"adjoint":true}}]]
[[0,{"TGate":{"basis":"Z","primed":false,"adjoint":false}}]]
[[0,{"TGate":{"basis":"X","primed":false,"adjoint":true}}]]
[[0,{"TGate":{"basis":"Z","primed":false,"adjoint":true}}]]
[[0,{"TGate":{"basis":"X","primed":false,"adjoint":true}}]]
[[0,{"TGate":{"basis":"Z","primed":false,"adjoint":false}}]]
[[0,{"TGate":{"basis":"X","primed":false,"adjoint":true}}]]
[[0,{"TGate":{"basis":"Z","primed":false,"adjoint":false}}]]
[[0,{"TGate":{"basis":"X","primed":false,"adjoint":true}}]]
[[0,{"TGate":{"basis":"Z","primed":false,"adjoint":true}}]]
[[0,{"TGate":{"basis":"X","primed":

### Circuit Benchmarking


Given a circuit consisting of Bicycle ISA gates, again in JSON list format, `bicycle_numerics` outputs a tally of statistics.

See the [Numerics Crate Readme](../crates/bicycle_numerics/README.md) for more information.

In [9]:
async def benchmark_compiled_circuit(circuit:str, # Bicycle ISA circuit as a json list
                                     qubits:int,
                                     code_:Literal["gross", "two-gross"],
                                     noise:Literal["1e-3", "1e-4"]):
    "Outputs a csv format tallying various statistics for each measurement."

    return await run_command("../target/release/bicycle_numerics",
                            str(qubits), f"{code_}_{noise}",
                             input_data=circuit)

In [10]:
compiled_circuit = await compile_pbc_circuit(test_circuit, code)
print(await benchmark_compiled_circuit(compiled_circuit, 12, code, "1e-4"))

code,p,i,qubits,idles,t_injs,automorphisms,measurements,joint_measurements,measurement_depth,end_time,total_error
gross,0.0001,1,22,51,86,26,22,1,15,21734,0.00007566432201584
gross,0.0001,2,22,2375,86,20,15,1,27,41668,0.00015132157664984
gross,0.0001,3,22,2318,86,12,10,1,36,61722,0.00022697378071616
gross,0.0001,4,22,0,0,8,9,0,44,61842,0.00022698287144456
gross,0.0001,5,22,0,0,6,7,0,51,61842,0.00022698994217296



In [11]:
compiled_circuit = await compile_pbc_circuit(transverse_field_ising_model(20, 5, 0.1, 0.1), code)
print(await benchmark_compiled_circuit(compiled_circuit, 20, code, "1e-4"))

code,p,i,qubits,idles,t_injs,automorphisms,measurements,joint_measurements,measurement_depth,end_time,total_error
gross,0.0001,1,22,123,91,20,16,1,15,22183,0.00008005326139112
gross,0.0001,2,22,2497,91,4,5,1,19,43382,0.0001600954152296
gross,0.0001,3,22,2482,91,14,16,1,34,64581,0.00024014868001768
gross,0.0001,4,22,2476,91,12,9,1,42,85780,0.00032019487431152
gross,0.0001,5,22,2452,91,8,16,1,57,106979,0.0004002481388136
gross,0.0001,6,22,2344,91,28,22,1,78,128178,0.00048030746388856
gross,0.0001,7,22,2308,91,16,20,1,97,149377,0.00056036476891168
gross,0.0001,8,22,2410,91,12,10,1,106,170576,0.00064041197311048
gross,0.0001,9,22,2395,91,20,22,1,127,191775,0.00072047129825888
gross,0.0001,10,22,2395,91,12,10,1,136,212974,0.00080051850243608
gross,0.0001,11,22,2545,91,40,38,1,157,236429,0.00088059398925728
gross,0.0001,12,22,0,91,20,16,0,171,259308,0.00096059915047128
gross,0.0001,13,22,0,91,4,3,0,174,280555,0.00104059118071408
gross,0.0001,14,22,0,91,14,14,0,188,303314,0.00112059432192808
